In [13]:
def getDataFromFcApi(symbol, period, dateFrom, dateTo):
    
    import requests
    import pandas as pd
    
    #print('symbol=' + symbol + ', period=' + period + ', dateFrom=' + dateFrom + ', dateTo=' + dateTo)

    url = "https://fcsapi.com/api-v2/forex/history?from=" + dateFrom
    url = url + '&to=' + dateTo
    url = url + '&symbol=' + symbol

    querystring = {
        "datatype" : "json",
        #"symbol" : symbol,
        #"from" : dateFrom,
        #"to" : dateTo,
        "period" : period,
        "access_key" : "zxyaA5VYrHr2iiujZ1cjuAnR5wwRAoFfXm6fTTpCL8keUg5W"
    }
    
    response = requests.request("GET", url, params=querystring)
    respJson = response.json()
    print(response.request.url)
    print('Response=' + respJson['msg'])
    #print(response.text)
    
    try:
        data = pd.DataFrame.from_dict(respJson["response"])
        #print(data.shape[0], ' elements')
        #print(data.head(1))
        return data    
    
    except BaseException as err:
        print(err)

def getDataFromInterval(symbol, intervalForQuery, dateTimeFrom, dateTimeTo):

    import pandas as pd
    import time
    from datetime import datetime
    from datetime import timedelta

    if (dateTimeTo is None):
        dateTo = datetime.now()
    else:
        dateTo = datetime.strptime(dateTimeTo, '%Y-%m-%d %H:%M:%S')

    if dateTimeFrom is None:
        dateTimeFrom = dateTo - timedelta(days=5)
    else:
        dateTimeFrom = datetime.strptime(dateTimeFrom, '%Y-%m-%d %H:%M:%S')
    
    print('Starting query from ' + dateTimeFrom.strftime('%d/%m/%Y %H:%M') + ' to ' + dateTo.strftime('%d/%m/%Y %H:%M'))
        
    interval = timedelta(minutes=300)
    totalRage = (int)((dateTo - dateTimeFrom).total_seconds() / 60)
    data = None
    execution = 0
    
    for x in range(1, totalRage, 300):
        
        diference = (int)((dateTo - dateTimeFrom).total_seconds() / 60)
        if diference > 299:
            diference = 299
            
        dateFrom = dateTo - timedelta(minutes=diference)
        retorno = getDataFromFcApi(symbol, intervalForQuery, 
                                    dateFrom.strftime('%Y-%m-%dT%H:%M'),
                                    dateTo.strftime('%Y-%m-%dT%H:%M'))
        if data is None:
            data = retorno
        else:
            data = data.append(retorno, ignore_index=True)
            print(data.shape[0], ' elements')
        
        dateTo = dateTo - interval
        execution += 1
        
        if execution == 3 and x+300<totalRage:
            time.sleep(60)
            execution = 1
        
    print('Process finish')
    return data